In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from yolov5 import detect

# Section A: Image Processing

def process_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 100, 200)
    
    plt.figure(figsize=(12, 4))
    plt.subplot(131), plt.imshow(gray, cmap='gray'), plt.title('Grayscale')
    plt.subplot(132), plt.imshow(blurred, cmap='gray'), plt.title('Blurred')
    plt.subplot(133), plt.imshow(edges, cmap='gray'), plt.title('Edges')
    plt.show()
    
    return gray, blurred, edges

# Section B: CNN for MNIST

def train_mnist_cnn():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
    
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.legend()
    plt.show()
    
    return model

# Section C: Object Detection

def detect_objects(image_path, model_path='yolov5s.pt'):
    results = detect.run(weights=model_path, source=image_path)
    return results

# Section D: Detect Triangle Inside Circle

def detect_shapes(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 3:  # Triangle
            cv2.drawContours(image, [approx], 0, (0, 255, 0), 2)
        elif len(approx) > 4:  # Circle or other shape
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            cv2.circle(image, (int(x), int(y)), int(radius), (255, 0, 0), 2)
    
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    
    return image


: 

In [ ]:
model = train_mnist_cnn()

Epoch 1/5
1875/1875 [==============================] - 16s 8ms/step - loss: 0.1520 - accuracy: 0.9549 - val_loss: 0.0664 - val_accuracy: 0.9784
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0522 - accuracy: 0.9844 - val_loss: 0.0530 - val_accuracy: 0.9814
Epoch 3/5
1875/1875 [==============================] - 11s 6ms/step - loss: 0.0326 - accuracy: 0.9906 - val_loss: 0.0495 - val_accuracy: 0.9823
Epoch 4/5
1875/1875 [==============================] - 12s 6ms/step - loss: 0.0220 - accuracy: 0.9929 - val_loss: 0.0420 - val_accuracy: 0.9858
Epoch 5/5
1875/1875 [==============================] - 10s 6ms/step - loss: 0.0160 - accuracy: 0.9951 - val_loss: 0.0488 - val_accuracy: 0.9849


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# Section A: Image Processing

def process_image(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 100, 200)
    
    plt.figure(figsize=(12, 4))
    plt.subplot(131), plt.imshow(gray, cmap='gray'), plt.title('Grayscale')
    plt.subplot(132), plt.imshow(blurred, cmap='gray'), plt.title('Blurred')
    plt.subplot(133), plt.imshow(edges, cmap='gray'), plt.title('Edges')
    plt.show()
    
    return gray, blurred, edges

# Section B: CNN for MNIST

def train_mnist_cnn():
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    x_train = x_train[..., np.newaxis]
    x_test = x_test[..., np.newaxis]
    
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))
    
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.legend()
    plt.show()
    
    return model

# Section C: Object Detection using OpenCV

def detect_objects_opencv(image_path, config_path, weights_path, names_path):
    net = cv2.dnn.readNetFromDarknet(config_path, weights_path)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    
    image = cv2.imread(image_path)
    height, width, _ = image.shape
    blob = cv2.dnn.blobFromImage(image, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)
    
    class_ids, confidences, boxes = [], [], []
    with open(names_path, 'r') as f:
        classes = [line.strip() for line in f.readlines()]
    
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x, center_y, w, h = (detection[:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                boxes.append([x, y, int(w), int(h)])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    for i in indices.flatten():
        x, y, w, h = boxes[i]
        label = f"{classes[class_ids[i]]}: {confidences[i]:.2f}"
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    
    return image

# Section D: Detect Triangle Inside Circle

def detect_shapes(image_path):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    for cnt in contours:
        approx = cv2.approxPolyDP(cnt, 0.02 * cv2.arcLength(cnt, True), True)
        if len(approx) == 3:  # Triangle
            cv2.drawContours(image, [approx], 0, (0, 255, 0), 2)
        elif len(approx) > 4:  # Circle or other shape
            (x, y), radius = cv2.minEnclosingCircle(cnt)
            cv2.circle(image, (int(x), int(y)), int(radius), (255, 0, 0), 2)
    
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    
    return image
